In [ ]:
import sqlite3
import requests
import pandas as pd
import time
import pytz
from datetime import datetime

In [ ]:
url_rodada_2023 = "https://api.sofascore.com/api/v1/unique-tournament/325/season/48982/events/round/38"

payload = {}

headers = {
  'authority': 'api.sofascore.com',
  'accept': '*/*' ,
  'accept-language': 'pt-BR,pt;q=0.9,en-US;q=0.8,en;q=0.7' ,
  'cache-control': 'max-age=0' ,
  'if-none-match': 'W/"660f6c17a5"' ,
  'origin': 'https://www.sofascore.com' ,
  'referer': 'https://www.sofascore.com/' ,
  'sec-ch-ua': 'Not A(Brand";v="99", "Opera GX";v="107", "Chromium";v="121"',
  'sec-ch-ua-mobile': '?0' ,
  'sec-ch-ua-platform': '"Windows"' ,
  'sec-fetch-dest': 'empty' ,
  'sec-fetch-mode': 'cors' ,
  'sec-fetch-site': 'same-site' ,
  'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36 OPR/107.0.0.0'
}

In [ ]:
df = pd.DataFrame()
for i in range(1, 39):
  url_rodada = f"https://api.sofascore.com/api/v1/unique-tournament/325/season/48982/events/round/{i}"
  response = requests.request("GET", url_rodada , headers = headers, data = payload)
  partidas = response.json()['events']
  df_temp = pd.json_normalize(partidas)
  df_temp = df_temp[['id',  'hasGlobalHighlights', 'crowdsourcingDataDisplayEnabled',  'startTimestamp', 'slug', 'finalResultOnly',
       'tournament.name', 'tournament.slug',
       'tournament.uniqueTournament.name',  'tournament.uniqueTournament.slug',  'tournament.uniqueTournament.userCount',  'tournament.uniqueTournament.id', 'tournament.id', 'season.name', 'season.year', 'season.editor',
       'season.id', 'roundInfo.round', 'status.code', 'status.description', 'status.type', 'homeTeam.name',  'homeTeam.slug', 'homeTeam.shortName',  'homeTeam.userCount', 'homeTeam.nameCode', 'homeTeam.disabled',
       'homeTeam.national', 'homeTeam.type', 'homeTeam.id', 'homeTeam.teamColors.primary', 'homeTeam.teamColors.secondary', 'homeTeam.teamColors.text', 'awayTeam.name','awayTeam.slug', 'awayTeam.shortName',
       'awayTeam.userCount', 'awayTeam.nameCode', 'awayTeam.disabled', 'awayTeam.national', 'awayTeam.type', 'awayTeam.id', 'awayTeam.teamColors.primary', 'awayTeam.teamColors.secondary', 'awayTeam.teamColors.text']]
  df_temp.rename(columns={df_temp.columns[0]: "game_id"}, inplace=True)
  df = pd.concat([df, df_temp])
  time.sleep(2)

  url_rodada = f"https://api.sofascore.com/api/v1/unique-tournament/325/season/58766/events/round/{i}"
  response = requests.request("GET", url_rodada , headers = headers, data = payload)
  partidas = response.json()['events']
  df_temp = pd.json_normalize(partidas)
  df_temp = df_temp[['id',  'hasGlobalHighlights', 'crowdsourcingDataDisplayEnabled',  'startTimestamp', 'slug', 'finalResultOnly','tournament.name', 'tournament.slug',
       'tournament.uniqueTournament.name',  'tournament.uniqueTournament.slug',  'tournament.uniqueTournament.userCount',  'tournament.uniqueTournament.id', 'tournament.id', 'season.name', 'season.year', 'season.editor',
       'season.id', 'roundInfo.round', 'status.code', 'status.description', 'status.type', 'homeTeam.name',  'homeTeam.slug', 'homeTeam.shortName',  'homeTeam.userCount', 'homeTeam.nameCode', 'homeTeam.disabled',
       'homeTeam.national', 'homeTeam.type', 'homeTeam.id', 'homeTeam.teamColors.primary', 'homeTeam.teamColors.secondary', 'homeTeam.teamColors.text', 'awayTeam.name','awayTeam.slug', 'awayTeam.shortName',
       'awayTeam.userCount', 'awayTeam.nameCode', 'awayTeam.disabled', 'awayTeam.national', 'awayTeam.type', 'awayTeam.id', 'awayTeam.teamColors.primary', 'awayTeam.teamColors.secondary', 'awayTeam.teamColors.text']]
  df_temp.rename(columns={df_temp.columns[0]: "game_id"}, inplace=True)
  df_temp['datetime'] = pd.to_datetime(df_temp['startTimestamp'], unit='s', utc=True).dt.tz_convert('UTC-03:00')
  df = pd.concat([df, df_temp])
  time.sleep(2)



In [ ]:
df.columns = df.columns.str.strip()

In [ ]:
conn = sqlite3.connect('/content/drive/MyDrive/projectDatabase/soccerDataLake.db')

In [ ]:
df.to_sql( 'games' , conn , if_exists='replace')

761

In [ ]:
query = "SELECT g.game_id FROM games g"
df = pd.read_sql_query(query, conn)
conn.close()
lista = df['game_id'].to_list()

In [ ]:
df = pd.DataFrame()
for i in lista:
  url_partida = f"https://api.sofascore.com/api/v1/event/{i}"
  response = requests.request("GET", url_partida , headers = headers, data = payload)
  partidas = response.json()['event']
  df_temp = pd.json_normalize(partidas)
  df_temp = df_temp[['id','slug','homeTeam.shortName', 'homeScore.period1', 'homeScore.period2', 'homeScore.normaltime', 'awayScore.period1', 'awayScore.period2' ,'awayScore.normaltime' ,'awayTeam.shortName',
                     'venue.id','venue.city.name', 'venue.stadium.name', 'venue.stadium.capacity', 'venue.country.alpha2',  'venue.country.alpha3', 'venue.country.name',
                     'referee.id','referee.name', 'referee.slug', 'referee.yellowCards', 'referee.redCards',  'referee.yellowRedCards', 'referee.games', 'referee.country.name']]
  df_temp.rename(columns={df_temp.columns[0]: "game_id"}, inplace=True)
  df = pd.concat([df, df_temp])
  time.sleep(2)

In [ ]:
df.columns.values

In [ ]:
score = df[['game_id','slug','homeTeam.shortName', 'homeScore.period1', 'homeScore.period2', 'homeScore.normaltime',
            'awayScore.period1', 'awayScore.period2' ,'awayScore.normaltime' ,'awayTeam.shortName']]

score.columns = df.columns.str.strip()
conn = sqlite3.connect("/content/drive/MyDrive/projectDatabase/sofascore.db")
score.to_sql( 'score' , conn , if_exists='replace')
conn.close()

In [ ]:
venue = df[['game_id', 'venue.id','venue.city.name', 'venue.stadium.name',
       'venue.stadium.capacity', 'venue.country.alpha2',
       'venue.country.alpha3', 'venue.country.name']]
venue.rename(columns={df_temp.columns[1]: "venue_id"} ,inplace=True)

venue.columns = df.columns.str.strip()
conn = sqlite3.connect("/content/drive/MyDrive/projectDatabase/sofascore.db")
venue.to_sql( 'venue' , conn , if_exists='replace')
conn.close()

In [ ]:
referee = df[['game_id','referee.id','referee.name',
       'referee.slug', 'referee.yellowCards', 'referee.redCards',
       'referee.yellowRedCards', 'referee.games', 'referee.country.name']]
referee.rename(columns={df_temp.columns[1]: "referee_id"} ,inplace=True)

referee.columns = df.columns.str.strip()
conn = sqlite3.connect("/content/drive/MyDrive/projectDatabase/soccerDataLake.db")
referee.to_sql( 'referee' , conn , if_exists='replace')
conn.close()

In [ ]:
df = pd.DataFrame()
for i in lista:
  url_partida = f"https://api.sofascore.com/api/v1/event/{i}/incidents"
  response = requests.request("GET", url_partida , headers = headers, data = payload)
  partidas = response.json()['incidents']
  df_temp = pd.json_normalize(partidas)
  df_temp['event_id'] = df_temp['id']
  df_temp['game_id'] = i
  df_temp = df_temp[['game_id', 'event_id', 'player.id', 'incidentType','incidentClass', 'reason', 'time', 'isHome' ]]
  df = pd.concat([df, df_temp])
  time.sleep(2)

df.columns = df.columns.str.strip()
conn = sqlite3.connect("/content/drive/MyDrive/projectDatabase/soccerDataLake.db")
df.to_sql( 'incidents' , conn , if_exists='replace')
conn.close()

In [ ]:
df = pd.DataFrame()
for i in lista:
  url = f"https://api.sofascore.com/api/v1/event/{i}/shotmap"
  response = requests.request("GET", url , headers = headers, data = payload)
  partidas = response.json()['shotmap']
  df_temp = pd.json_normalize(partidas)
  df_temp['shot_id'] = df_temp['id']
  df_temp['player_id'] = df_temp['player.id']
  df_temp['game_id'] = i
  df_temp = df_temp[['game_id', 'shot_id', 'player_id', 'playerCoordinates.x','playerCoordinates.y', 'incidentType', 'bodyPart', 'situation', 'shotType', 'time', 'addedTime', 'goalMouthLocation', 'xg', 'xgot' ]]
  df = pd.concat([df, df_temp])
  time.sleep(2)

df.columns = df.columns.str.strip()
conn = sqlite3.connect("/content/drive/MyDrive/projectDatabase/soccerDataLake.db")
df.to_sql( 'playerShotmap' , conn , if_exists='replace')
conn.close()

In [ ]:
df = pd.DataFrame()
for i in lista:
    url_partida = f"https://api.sofascore.com/api/v1/event/{i}/statistics"
    response = requests.request("GET", url_partida , headers = headers, data = payload)
    grupos = response.json()['statistics']
    df_temp = pd.json_normalize(grupos)
    df_temp['game_id'] = i
    df_temp = df_temp.apply(pd.Series.explode)
    df_temp = df_temp.reset_index(drop=True)
    df_temp = df_temp.join(pd.json_normalize(df_temp['groups']))
    df_temp.drop('groups', axis= 1, inplace=True)
    df_temp = df_temp.apply(pd.Series.explode)
    df_temp = df_temp.join(pd.json_normalize(df_temp['statisticsItems']))
    df_temp.drop('statisticsItems', axis=1, inplace = True)
    df = pd.concat([df, df_temp])
    time.sleep(2)

df.columns = df.columns.str.strip()
conn = sqlite3.connect("/content/drive/MyDrive/projectDatabase/soccerDataLake.db")
df.to_sql( 'gameStats' , conn , if_exists='replace')
conn.close()

In [ ]:
df = pd.DataFrame()
for i in lista:
    url_partida = f"https://api.sofascore.com/api/v1/event/{i}/average-positions"
    response = requests.request("GET", url_partida , headers = headers, data = payload)
    grupos  = response.json()
    df_home = pd.json_normalize(response.json()['home'])
    df_home['side'] = 'home'

    df_away = pd.json_normalize(response.json()['away'])
    df_away['side'] = 'away'

    df_temp = pd.concat([df_home,df_away])
    df_temp['game_id'] = i

    df_temp = df_temp.rename(columns = {'player.id':'player_id'})
    df_temp = df_temp[['game_id', 'player_id', 'side','averageX', 'averageY', 'pointsCount', 'player.name', 'player.slug', 'player.shortName', 'player.position', 'player.jerseyNumber']]
    df = pd.concat([df, df_temp])
    time.sleep(2)

df.columns = df.columns.str.strip()
conn = sqlite3.connect("/content/drive/MyDrive/projectDatabase/soccerDataLake.db")
df.to_sql( 'lineup' , conn , if_exists='replace')
conn.close()

In [ ]:
df = pd.DataFrame()
for i in lista:
    url_partida = f"https://api.sofascore.com/api/v1/event/{i}/average-positions"
    response = requests.request("GET", url_partida , headers = headers, data = payload)
    grupos  = response.json()
    df_home = pd.json_normalize(response.json()['home'])
    df_home['side'] = 'home'

    df_away = pd.json_normalize(response.json()['away'])
    df_away['side'] = 'away'

    df_temp = pd.concat([df_home,df_away])
    df_temp['game_id'] = i

    df_temp = df_temp.rename(columns = {'player.id':'player_id'})
    df_temp = df_temp[['game_id', 'player_id', 'side','averageX', 'averageY', 'pointsCount', 'player.name', 'player.slug', 'player.shortName', 'player.position', 'player.jerseyNumber']]
    df = pd.concat([df, df_temp])
    time.sleep(2)

df.columns = df.columns.str.strip()
conn = sqlite3.connect("/content/drive/MyDrive/projectDatabase/soccerDataLake.db")
df.to_sql( 'lineup' , conn , if_exists='replace')
conn.close()

In [ ]:
query = "SELECT g.game_id FROM lineup g"
df = pd.read_sql_query(query, conn)
conn.close()
lista = df['game_id'].to_list()

query = "SELECT g.player_id FROM lineup g"
df = pd.read_sql_query(query, conn)
conn.close()
jogadores = df['player_id'].to_list()

columns_to_drop = ['team.shortName', 'team.gender', 'team.sport.name', 'team.sport.slug', 'team.sport.id', 'team.userCount','team.teamColors.primary', 'team.teamColors.secondary','team.teamColors.text',
       'player.fieldTranslations.nameTranslation.ar','player.fieldTranslations.shortNameTranslation.ar','team.nameCode', 'team.disabled', 'team.national', 'team.type','player.dateOfBirthTimestamp']

df = pd.DataFrame()
for i,j in zip(lista,jogadores):
    url_partida = f"https://api.sofascore.com/api/v1/event/{i}/player/{j}/statistics"
    response = requests.request("GET", url_partida , headers = headers, data = payload)
    grupos  = response.json()
    df_temp = pd.json_normalize(grupos)
    df_temp = df_temp.rename(columns = {'player.id':'player_id'})
    df_temp['game_id'] = i
    df_temp['playerBirthday'] = pd.to_datetime(df_temp['player.dateOfBirthTimestamp'], unit='s', utc=True).dt.tz_convert('UTC-03:00')
    df_temp = df_temp.drop(columns_to_drop, axis=1)
    df = pd.concat([df, df_temp])
    time.sleep(2)

df.columns = df.columns.str.strip()
conn = sqlite3.connect("/content/drive/MyDrive/projectDatabase/soccerDataLake.db")
df.to_sql( 'playerStats' , conn , if_exists='replace')
conn.close()

In [ ]:
df_temp.columns.values

array(['position', 'player.name', 'player.firstName', 'player.lastName',
       'player.slug', 'player.shortName', 'player.position',
       'player.jerseyNumber', 'player.userCount', 'player.id',
       'player.marketValueCurrency', 'player.dateOfBirthTimestamp',
       'player.fieldTranslations.nameTranslation.ar',
       'player.fieldTranslations.shortNameTranslation.ar', 'team.name',
       'team.slug', 'team.shortName', 'team.gender', 'team.sport.name',
       'team.sport.slug', 'team.sport.id', 'team.userCount',
       'team.nameCode', 'team.disabled', 'team.national', 'team.type',
       'team.id', 'team.teamColors.primary', 'team.teamColors.secondary',
       'team.teamColors.text',
       'team.fieldTranslations.nameTranslation.ru',
       'statistics.totalPass', 'statistics.accuratePass',
       'statistics.totalLongBalls', 'statistics.accurateLongBalls',
       'statistics.goalAssist', 'statistics.totalCross',
       'statistics.accurateCross', 'statistics.aerialLost',
     